In [1]:
# initializing otter-grader
import otter
grader = otter.Notebook()

# Lab 7: Simple Linear Regression

### Objective

In this lab, we'll review some of the details of how linear regression works. 

We will also show how to do linear regression using various real world tools including Altair's `transform_regression`, `scipy.optimize`, and `scikit-learn`. In real world data science work, you are far more likely to use something similar to the former (`transform_regression`) and latter (`scikit-learn`) approaches, but it's important to know how to use the formulaic and the `scipy.optimize` approaches so that you understand what's really going on.

**This assignment should be completed and submitted before 11:59 PM on Friday, May 15, 2020.**


### Collaboration Policy

Data science is a collaborative activity. While you may talk to others about the labs, we ask that you **write your solutions individually**. If you do discuss the assignments with others, please **include their names** in the following cell:

_List collaborators here_

In [2]:
# Run this cell
import pandas as pd
import numpy as np
import altair as alt
import seaborn as sns

We begin by importing the tips dataset from seaborn's dataset bank.

In [3]:
tips = sns.load_dataset("tips")

In [4]:
tips.head(5)

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In this lab, we're interested in studying the **relationship between two variables**. Specifically, we're interested in the relationship between the `total_bill` column and `tip` column. Our goal will be to predict tips ($Y$) from total_bill ($x$). In other words, we want to find values of $\alpha$ and $\beta$ so that given the value of $x$, we can predict $Y$
$$\boxed{Y = \alpha + \beta x}$$
We will now explore different ways to obtain the optimal values of $\alpha, \beta$, called $\hat{\alpha}, \hat{\beta}$, where $\hat{Y} = \hat{\alpha} + \hat{\beta}x$.

First, let's build a boilerplate regression plot in altair, which will both provide a scatterplot of `tip` vs `total_bill` and also display the least-squares line of best fit. This method of creating a regression line is great for quickly calculating the regression line when doing data analysis; however, in this lab, we will look a bit deeper into this line. This line of best fit is what we will look to determine empirically in three different ways: manually using the formula from lecture, `scipy.optimize`, and `scikit-learn`. 

In [5]:
chart = alt.Chart(tips).mark_point().encode(
    x='total_bill',
    y='tip'    
)
chart + chart.transform_regression('total_bill', 'tip').mark_line(color='red')

alt.LayerChart(...)

## Question 1 – Manual Formulation

Recall the following expression for the line of best fit.

$$\hat{y_i} = \bar{y} + r \frac{SD(y)}{SD(x)} (x_i - \bar{x})$$

where $\bar{x}$, $\bar{y}$, $SD(x)$, $SD(y)$ correspond to the means and standard deviations of $x$ and $y$, respectively, and $r$ is the correlation coefficient.

### Question 1a

Assign `xbar`, `ybar`, `stdx`, `stdy`, and `r`, such that they align with our dataset.

- Hint: Remember, in our case, `y` is `tip`, and `x` is `total_bill`.
- Hint: Use `np.corrcoef` to compute `r`. Note that the output of `np.corrcoef` is a **matrix, not a number**, so you'll need to collect the correlation coefficient by **indexing into the returned array**. 

<!--
BEGIN QUESTION
name: q1a
points: 5
manual: false
-->

In [12]:
xbar = np.mean(tips['total_bill'])
ybar = np.mean(tips['tip'])
stdx = np.std(tips['total_bill'])
stdy = np.std(tips['tip'])
r = np.corrcoef(tips['total_bill'], tips['tip'])[0,1]

### Question 1b

Now, set `b_hat` and `a_hat` correctly, in terms of the variables you defined above. Remember that `b_hat` and `a_hat` are our regression line's intercept and slope, respectively.

- Hint: Try and match the slope and intercept in $\hat{y_i} = \hat{\alpha} + \hat{\beta}x_i$ to the slope and intercept in $\hat{y_i} = \bar{y} + r \frac{SD(y)}{SD(x)} (x_i - \bar{x})$.

- Hint: You may want to define `a_hat` in terms of `b_hat` (i.e., solve for `b_hat` and then use it to find `a_hat`).

<!--
BEGIN QUESTION
name: q1b
points: 3
manual: false
-->

In [16]:
b_hat

4.347714207346626

In [22]:
b_hat = r*(stdy/stdx)
a_hat = ybar - b_hat*(xbar)

### Question 1c

Now, use `a_hat` and `b_hat` to predict the tip for a total bill amount of \$20. Store your result in `predicted_20`.

<!--
BEGIN QUESTION
name: q1c
points: 2
manual: false
-->

In [24]:
predicted_20 = b_hat*(20) + a_hat
predicted_20

3.0207599612417404

### Question 1d
Define `regression` as a `pd.Series` containg predicted values of $Y$ values (i.e., predicted `"tip"` values) for the observed values of total bills (`tips["total_bill"]`). You will need to use `a_hat`, `b_hat`, and `tips["total_bill"]`.

<!--
BEGIN QUESTION
name: q1d
points: 2
manual: false
-->

In [26]:
regression = tips['total_bill']*b_hat + a_hat

### Question 1e

Now, create the scatter plot of `tip` vs `total_bill`, along with the regression line you just calculated on the same plot.

* **Hint:** You'll need to create a new dataframe to plot the regression line 
* **Hint:** You will want the y axis to be the regression values you calculated in the last part

<!--
BEGIN QUESTION
name: q1e
points: 4
manual: true
-->
<!-- EXPORT TO PDF -->

In [30]:
tips['regression'] = regression
tips['regression']

0      2.704636
1      2.006223
2      3.126835
3      3.407250
4      3.502822
         ...   
239    3.969131
240    3.774836
241    3.301175
242    2.791807
243    2.892630
Name: regression, Length: 244, dtype: float64

In [35]:
chart1 = alt.Chart(tips).mark_point().encode(
    alt.Y('tip'),
    alt.X('total_bill'))

chart2 = alt.Chart(tips).mark_line().encode(
    alt.Y('regression'),
    alt.X('total_bill'))

chart1 + chart2

alt.LayerChart(...)

### Question 1f

Consider $r$, the correlation coefficient between `tips` and `total_bill`.

In [36]:
r

0.6757341092113641


**In the cell below**, comment on the value of $r$, and what it means in the context of the above scatter plot.
<!--
BEGIN QUESTION
name: q1f
points: 3
manual: true
-->
<!-- EXPORT TO PDF -->

There is a sizeable linear relationship between tips and total_bill, but the relationship cannot be completely explained by linearity. As total_bill increases, tips tends to generally increase by a bigger amount proportionally (to itself). 

## Question 2 – Using Scipy Minimize

`scipy.minimize` is a powerful method that can determine the optimal value of a variety of different functions. In practice, it is used to minimize functions that have no (or difficult to obtain) analytical solutions (it is a **numerical method**).

It is overkill for our simple example, but nonetheless, we will show you how to use it, as it will become useful in the near future.

### Question 2a

Firstly, fill out the definition of `L2_tip_loss` so that it computes the empirical risk for a given choice of $\alpha$ and $\beta$. That is, it computes

$$\frac{1}{n} \sum_{i = 1}^n(y_i - (\alpha + \beta x_i))^2$$

where, again, $x$ and $y$ refer to `"total_bill"` and `"tip"`.

<!--
BEGIN QUESTION
name: q2a
points: 3
manual: false
-->

In [66]:
def L2_tip_loss(a, b):
    n = len(tips)
    y = (tips['tip'] - (a + b*tips['total_bill']))**2
    return (1/n)*(np.sum(y))

Try out different `a` and `b` values. Observe that if you pick values close to `a_hat` and `b_hat` then the loss is lower. 

In fact, in linear regression, we are trying to minimize the sum of squares error when picking the slope `a_hat` and intercept `b_hat` of the regression line. We are able to calculate it directly using the formulas in question 1, but had our model required the minimization of a function that had no analytical solution, we would be using this function to approximate the values needed to minimize our loss.

In [67]:
L2_tip_loss(0.9, 0.1)

1.052336405737705

The `minimize` function we saw in Lab 2 can also minimize functions of multiple variables. There's one quirk, however, which is that the function has to **accept its parameters as a single list**.

For example, consider the multivariate $f(u, v) = u^2 - 2 u v - 3 v + 2 v^2$. It turns out this function's minimum is at $(1.5, 1.5)$. To minimize this function, we create `f`.

In [68]:
def f(theta):
    u = theta[0]
    v = theta[1]
    return u**2 - 2 * u * v - 3 * v + 2 * v**2

In [69]:
from scipy.optimize import minimize
minimize(f, x0 = [0.0, 0.0]) 

# As an aside: x0 is the "initial guess" for the optimal theta. minimize works iteratively.

      fun: -2.2499999999999982
 hess_inv: array([[0.99999999, 0.5       ],
       [0.5       , 0.5       ]])
      jac: array([-5.96046448e-08,  0.00000000e+00])
  message: 'Optimization terminated successfully.'
     nfev: 16
      nit: 3
     njev: 4
   status: 0
  success: True
        x: array([1.49999995, 1.49999997])

### Question 2b

Define `L2_tip_loss_list` which is exactly like `L2_tip_loss` except that it takes in **a single list of 2 variables** rather than two separate variables. For example `L2_tip_loss_list([2, 3])` should return the same value as `L2_tip_loss(2, 3)`.

<!--
BEGIN QUESTION
name: q2b
points: 2
manual: false
-->

In [70]:
def L2_tip_loss_list(variablez):
    n = len(tips)
    y = (tips['tip'] - (variablez[0] + variablez[1]*tips['total_bill']))**2
    return (1/n)*(np.sum(y))

In [64]:
L2_tip_loss_list([0.9,0.1])

1.052336405737705

### Question 2c

Now, set `minimized` to the result of calling `minimize` to optimize this loss function.

- Hint: Make sure to set `x0`. (Try using the origin.)

<!--
BEGIN QUESTION
name: q2c
points: 2
manual: false
-->

In [72]:
minimized = minimize(L2_tip_loss_list, x0 = [0.0, 0.0])

Let's look at the output of your call to `minimize`.

In [75]:
minimized

      fun: 1.0360194420115871
 hess_inv: array([[ 2.98000083, -0.12534155],
       [-0.12534155,  0.00633488]])
      jac: array([7.45058060e-08, 5.96046448e-08])
  message: 'Optimization terminated successfully.'
     nfev: 20
      nit: 3
     njev: 5
   status: 0
  success: True
        x: array([0.92027066, 0.10502447])

The following cell will print out the values of `a_hat` and `b_hat` computed from both methods ("manual" refers to the technique in Question 1). If you've done everything correctly, these should be very close to one another.

In [76]:
print('a_hat_scipy: ', minimized['x'][0])
print('a_hat_manual: ', a_hat)
print('\n')
print('b_hat_scipy: ', minimized['x'][1])
print('b_hat_manual: ', b_hat)

a_hat_scipy:  0.9202706617668626
a_hat_manual:  0.9202696135546735


b_hat_scipy:  0.10502446590151857
b_hat_manual:  0.10502451738435335


The reason these don't match past the first 5 decimal places is due to the fact that `scipy.minimize` is a numerical method, meaning it approximates the optimal value using some sort of non-algebraic procedure. For our purposes, though, these values are essentially the same.

## Question 3 – Using Scikit Learn

Yet another way to fit a linear regression model is to use scikit learn, an industry standard package for machine learning applications. 

To do so, we first create a `LinearRegression` object.

In [77]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

Here, `model` is like a "blank slate" for a linear model. Now, we need to tell `model` to "fit" itself to the data. Essentially, this is doing exactly what you did in the previous part of this lab (creating a loss function and finding the parameters that minimize that loss).

<i>Note: `X` needs to be a matrix (or DataFrame), as opposed to a single array (or Series). This is because `sklearn.linear_model` is robust enough to be used for multiple regression.</i>

In [78]:
model.fit(X = tips[['total_bill']], y= tips['tip'])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

Now that the model exists, we can look at the `a_hat` and `b_hat` values it found, which are given in the attributes `intercept` and `coef`, respectively.

In [79]:
b_hat_sklearn = model.coef_

In [80]:
a_hat_sklearn = model.intercept_

In [81]:
print('a_hat_scipy: ', minimized['x'][0])
print('a_hat_manual: ', a_hat)
print('\n')
print('b_hat_scipy: ', minimized['x'][1])
print('b_hat_manual: ', b_hat)

a_hat_scipy:  0.9202706617668626
a_hat_manual:  0.9202696135546735


b_hat_scipy:  0.10502446590151857
b_hat_manual:  0.10502451738435335


To use the `scikit-learn` linear regression model to make predictions, you can use the `model.predict` method:

In [91]:
np.array(20).reshape(-1,1).shape

(1, 1)

In [82]:
model.predict(np.array(20).reshape(-1,1))[0]

3.0207599612417404

The above line of code tells us that `model` predicts a tip of \$3.02 given a total bill amount of 20 dollars. This is the same as doing `a_hat + b_hat * 20` as in Question 1c.

### Question 3a

Create a linear regression plot using `model.predict`. It should look very similar (if not the same) as your plot from Question 1d.

* **Hint**: Use `np.linspace()` to create your x-axis

<!--
BEGIN QUESTION
name: q3a
points: 5
manual: true
-->
<!-- EXPORT TO PDF -->

In [92]:
tips['total_bill'][0].shape

()

In [139]:
y_tips = tips['tip'].sort_values()
y_tips

67      1.00
236     1.00
92      1.00
111     1.00
0       1.01
       ...  
141     6.70
59      6.73
23      7.58
212     9.00
170    10.00
Name: tip, Length: 244, dtype: float64

In [148]:
tips['total_bill'].max()

50.81

In [152]:
x = np.linspace(0,55, num=len(tips))
y_tips = tips['tip']
y_fit = model.predict(np.linspace(0,tips['total_bill'].max(), num=len(tips)).reshape(-1,1))
df = pd.DataFrame({'x':x, 'y_tips':y_tips, 'y_fit':y_fit})

In [154]:
chart1 = alt.Chart(df).mark_point().encode(
    alt.Y('y_tips'),
    alt.X('x'))

chart2 = alt.Chart(df).mark_line().encode(
    alt.Y('y_fit'),
    alt.X('x'))

chart1 + chart2

alt.LayerChart(...)

## Question 4 - Graduate Admissions

Using `pd.read_csv()`, load the dataset `"admissions.csv"`. This dataset was added to [Kaggle](https://www.kaggle.com/mohansacharya/graduate-admissions) "with the purpose of helping students in shortlisting universities with their profiles."

We would like yo to calculate the regression line of `GRE Score` to `Chance of Admit` using any of the methods discussed in this lab. Then display a plot showing the data and the regression line that you calculated.

In [159]:
admissions = pd.read_csv("admissions.csv")
df = pd.DataFrame(admissions)
df.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,324,107,4,4.0,4.5,8.87,1,0.76
2,3,316,104,3,3.0,3.5,8.00,1,0.72
3,4,322,110,3,3.5,2.5,8.67,1,0.80
4,5,314,103,2,2.0,3.0,8.21,0,0.65


In [ ]:
### CALCULATE YOUR REGRESSION LINE

In [169]:
### PLOT YOUR REGRESSION LINE
chart = alt.Chart(df).mark_point().encode(
    alt.X('GRE Score', scale=alt.Scale(domain=(df['GRE Score'].min(), df['GRE Score'].max()))),
    alt.Y('Chance of Admit', scale=alt.Scale(domain=(df['Chance of Admit'].min(), df['Chance of Admit'].max())))   
)
chart + chart.transform_regression('GRE Score', 'Chance of Admit').mark_line(color='red')

alt.LayerChart(...)

### Question 4b - Communicate Your Results

Explain in the cell below what your regression line means.

GRE SCORE is directly proportional to Chance of Admit, highly correlated. 

### Question 4 BONUS

Answer all the questions in the cell below to get bonus points
* Why is linear regression not the best estimate for the admissions chance variable that we see in this dataset, and specifically where does our regression line break down?
* What is another regression model that would more accurately fit the domain of values seen in the `Chance of Admit` column? 

The data doesn't have a completely linear relationship throughout. It breaks down at low values of GRE, where some other outstanding feature of a candidate may potentially overshadow a low GRE score

Quadratic regression could work better. 

**Congrats!** You are finished with this assignment.

# Submit
Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output.
**Please save before submitting!**

# Running Built-in Tests
1. All tests are in `tests` directory
1. Each python file in `tests` is a test
1. `grader.check('testname')` runs test `'testname'`, e.g. `'q1'`
1. `grader.check_all()` runs all visible tests

In [171]:
# Run built-in checks
grader.check_all()

In [172]:
# Generate pdf in classic notebook (does not work in JupyterLab)
import nb2pdf
nb2pdf.convert('lab07.ipynb')

# To generate pdf using command-line, run in terminal,
# nb2pdf lab07.ipynb

<IPython.core.display.Javascript object>

# Submission Checklist
1. Check filename is 'lab07.ipynb'
1. Save file to confirm all changes are on disk
1. Run *Kernel > Restart & Run All* to execute all code from top to bottom
1. Check `grader.check_all()` output
1. Save file again to write any new output to disk
1. Check generated pdf that all responses are displayed correctly
1. Submit to Gradescope